## trans annos from xml to txt

In [1]:
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """

    tree = ET.parse(filename)
    objects = []

    for pic in tree.findall('size'):
        pic_struct = {}
        pic_struct['width'] = int(pic.find('width').text)
        pic_struct['height'] = int(pic.find('height').text)
        objects.append(pic_struct)

    for obj in tree.findall('object'):
        # skip D30
        if obj.find('name').text == 'D30':
            continue
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(float(bbox.find('xmin').text)),
                              int(float(bbox.find('ymin').text)),
                              int(float(bbox.find('xmax').text)),
                              int(float(bbox.find('ymax').text))]
        objects.append(obj_struct)
    return objects

In [2]:
import os, sys
import xml.etree.ElementTree as ET

ROOT_DIR = '/home/wangwenzhe/tf-faster-rcnn-rddc'
VOCDEVKIT_DIR = os.path.join(ROOT_DIR, 'data', 'VOCdevkit2007')

VOCopts_classes=['D00', 'D01', 'D10', 'D11', 'D20', 'D40', 'D43', 'D44']
years = ['2007']
split = ['trainval', 'test']
fileName = 'window_file_voc2007_trainval.txt'
fid = open(fileName, 'wt')

# just running
print('VOC years: %s' %(years[0]))
txt = os.path.join(VOCDEVKIT_DIR, 'VOC' + years[0], 'ImageSets', 'Main' , split[0] + '.txt')
f = open(txt); idList = f.readlines(); f.close()
print('Trainval images number: %s' %(len(idList)))

for u in range(len(idList)):
    nm = idList[u].strip() + '.jpg'
    xm = idList[u].strip() + '.xml'
    imgFullName = os.path.join(VOCDEVKIT_DIR, 'VOC' + years[0], 'JPEGImages', nm)
    xmlPath = os.path.join(VOCDEVKIT_DIR, 'VOC' + years[0], 'Annotations', xm)
    nObjs = parse_rec(xmlPath)
    
    fid.write('%s\n' % imgFullName)
    fid.write('%d\n' %(len(nObjs) - 1))
    
    for j in range(1, len(nObjs)):
        clsIds = nObjs[j]['name']
        xmin, ymin, xmax, ymax = nObjs[j]['bbox']
        if (xmax <= xmin) | (ymax <= ymin):
            print('Warning. Confirm xmin <= xmax and ymin <= ymax[%d <= %d and %d <= %d]' %(xmin, xmax, ymin, ymax))
        clsId = VOCopts_classes.index(clsIds) + 1
        t_lines = str(clsId) + ' ' + str(xmin) + ' ' + str(ymin) + ' ' + str(xmax) + ' ' + str(ymax)
        fid.write('%s\n' % t_lines)
    
print('Trans annotations finished.')
fid.close()

VOC years: 2007
Trainval images number: 7240
Trans annotations finished.
